In [2]:
from models.rnn.rnn_plain import RNN_Plain
from models.rnn.rnn_attention import RNN_Attention

from lang import load_data_int_seq
from utils import accuracy_score
%load_ext autoreload
%autoreload 2

In [4]:
output_lang, input_lang, train, X_test, y_test = load_data_int_seq()

In [5]:
## FOR DATALOADER
# output_lang, input_lang, train_data_iterable, test_data_iterable = load_data_int_seq()

## Plain RNN

In [6]:
algo = RNN_Plain(symbols = "+*-0123456789t", 
output_sequence_length = 9, 
encoded_seq_length = 9, 
num_epochs = 500, 
input_size = input_lang.n_words, 
hidden_size = 256, 
output_size=output_lang.n_words, 
embedding_size = 28, 
batch_size = 32, 
num_gru_layers = 2,
dropout_prob = 0.3,
calc_magnitude_on=False)

# algo.__class__ = RNN_Plain


EncoderRNN(
  (embedding): Embedding(2564, 28)
  (gru): GRU(28, 256, num_layers=2, dropout=0.3)
)
DecoderRNN(
  (embedding): Embedding(16, 28)
  (gru): GRU(28, 256, num_layers=2, dropout=0.3)
  (out): Linear(in_features=256, out_features=16, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [7]:
# algo.train(input_lang, output_lang, train_data_iterable) 
algo.train(input_lang, output_lang, train)

# from line_profiler import LineProfiler

# lp = LineProfiler()
# lp_wrapper = lp(algo.train)
# lp_wrapper(input_lang, output_lang, train)
# lp.print_stats()


0m 6s (- 1m 1s) (50 10%) 2.5351
0m 12s (- 0m 48s) (100 20%) 2.4355
0m 17s (- 0m 40s) (150 30%) 2.3873
0m 22s (- 0m 33s) (200 40%) 2.3256
0m 27s (- 0m 27s) (250 50%) 2.2763
0m 32s (- 0m 21s) (300 60%) 2.2553
0m 36s (- 0m 15s) (350 70%) 2.2380
0m 41s (- 0m 10s) (400 80%) 2.2009
0m 46s (- 0m 5s) (450 90%) 2.2070
0m 50s (- 0m 0s) (500 100%) 2.1828


In [8]:
pred = algo.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

RuntimeError: Expected hidden size (2, 1, 256), got [2, 32, 256]

In [ ]:
accuracy_score(pred, y_test[:1000])

### Plain RNN with Magnitude loss

In [ ]:
magn_algo = RNN_Plain(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=True)
magn_algo.train(input_lang, output_lang, train)

In [ ]:
pred = magn_algo.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

##  Attention-based

In [ ]:
algo_attn = RNN_Attention(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=False)
algo_attn.train(input_lang, output_lang, train)

In [ ]:
pred = algo_attn.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

### Attention with magnitude loss

In [ ]:
algo_attn_magn = RNN_Attention(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=True)
algo_attn_magn.train(input_lang, output_lang, train)

In [ ]:
pred = algo_attn_magn.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

In [4]:
import torch
import torch.nn as nn
m = nn.LogSoftmax()
input = torch.randn(2, 3)
print(input)
output = m(input)
print(output)

In [42]:
# 2D loss example (used, for example, with image inputs)
N, C = 5, 4
loss = nn.NLLLoss()

# input is of size N x C x height x width
data = torch.randn(N, 16, 10, 10)
conv = nn.Conv2d(16, C, (3, 3))
m = nn.LogSoftmax(dim=1)

# print(data.shape)
activated = m(conv(data))
print("output: ", activated.shape)

# each element in target has to have 0 <= value < C
target = torch.empty(N, 8, 8, dtype=torch.long).random_(0, C)
print("target: ", target.shape)
output = loss(activated, target)

# output.backward()

In [41]:
m = nn.LogSoftmax(dim=1)
loss = nn.NLLLoss()
# input is of size N x C = 3 x 5
# input = minibatchsize, number of categories

input = torch.randn(3, 5, requires_grad=True)
# print(input.shape)
activation = m(input)
print(activation.shape)
# each element in target has to have 0 <= value < C
target = torch.tensor([1, 0, 4])
print(target.shape)
output = loss(activation, target)
# output.backward()